Sequential Quadratic Programming

Dependencies: `jax`, `scipy`, `numpy`

In [13]:
import jax
import jax.numpy as jnp
from jax import jacfwd, jacrev, hessian

from scipy.optimize import minimize

import numpy as np

Define your own Objective Functions and Constrains here :

(we use rosenbrock function here as an example)

$f(x)=\sum^{N-1}_{i=1}100(x_{i+1}-{x_i}^2)^2+(1-x_i)^2$

In [2]:
#the rosenbrock function
def f(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [3]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

key = random.PRNGKey(0)

In [4]:
from jax import jacfwd, jacrev

def sigmoid(x):
    return 0.5 * (jnp.tanh(x / 2) + 1)

# Outputs probability of a label being true.
def predict(W, b, inputs):
    return sigmoid(jnp.dot(inputs, W) + b)

# Build a toy dataset.
inputs = jnp.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = jnp.array([True, True, False, True])

# Training loss is the negative log-likelihood of the training examples.
def loss(W, b):
    preds = predict(W, b, inputs)
    label_probs = preds * targets + (1 - preds) * (1 - targets)
    return -jnp.sum(jnp.log(label_probs))

# Initialize random model coefficients
key, W_key, b_key = random.split(key, 3)
W = random.normal(W_key, (3,))
b = random.normal(b_key, ())


# Isolate the function from the weight matrix to the predictions
f = lambda W: predict(W, b, inputs)

J = jacfwd(f)(W)
print("jacfwd result, with shape", J.shape)
print(J)

J = jacrev(f)(W)
print("jacrev result, with shape", J.shape)
print(J)

def hessian(f):
    return jacfwd(jacrev(f))

H = hessian(f)(W)
print("hessian, with shape", H.shape)
print(H)

print("W: ",W)

jacfwd result, with shape (4, 3)
[[ 0.05981758  0.12883787  0.08857603]
 [ 0.04015916 -0.04928625  0.00684531]
 [ 0.12188288  0.01406341 -0.3047072 ]
 [ 0.00140431 -0.00472531  0.00263782]]
jacrev result, with shape (4, 3)
[[ 0.05981757  0.12883787  0.08857603]
 [ 0.04015916 -0.04928625  0.00684531]
 [ 0.12188289  0.01406341 -0.3047072 ]
 [ 0.00140431 -0.00472531  0.00263782]]
hessian, with shape (4, 3, 3)
[[[ 0.02285465  0.04922541  0.03384247]
  [ 0.04922541  0.10602397  0.07289147]
  [ 0.03384247  0.07289147  0.05011288]]

 [[-0.03195215  0.03921401 -0.00544639]
  [ 0.03921401 -0.04812629  0.00668421]
  [-0.00544639  0.00668421 -0.00092836]]

 [[-0.01583708 -0.00182736  0.03959271]
  [-0.00182736 -0.00021085  0.00456839]
  [ 0.03959271  0.00456839 -0.09898177]]

 [[-0.00103524  0.00348343 -0.00194457]
  [ 0.00348343 -0.01172127  0.0065432 ]
  [-0.00194457  0.0065432  -0.00365263]]]
W:  [-0.36838785 -2.275689    0.01144757]


In [5]:
from jax import jacfwd, jacrev

def f(x):
    return jnp.power(x,3).sum()
print(f(jnp.array([1.,2.,3.])))

def hessian(f):
    return jacfwd(jacrev(f))

def jacobian(f):
    return jacfwd(f)

J = jacobian(f)(jnp.array([1.,2.,3.]))
print("jacobian, with shape", J.shape)
print(J)

H = hessian(f)(jnp.array([1.,2.,3.]))
print("hessian, with shape", H.shape)
print(H)

36.0
jacobian, with shape (3,)
[ 3. 12. 27.]
hessian, with shape (3, 3)
[[ 6.  0.  0.]
 [ 0. 12.  0.]
 [ 0.  0. 18.]]


In [6]:
import numpy as np

from scipy.optimize import minimize

from scipy.optimize import Bounds

bounds = Bounds([0, -0.5], [1.0, 2.0])

def rosen(x):

    """The Rosenbrock function"""

    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def hessian(f):
    return jacfwd(jacrev(f))

def jacobian(f):
    return jacfwd(f)

rosen_der=jacobian(rosen)


ineq_cons = {'type': 'ineq',

             'fun' : lambda x: np.array([1 - x[0] - 2*x[1],

                                         1 - x[0]**2 - x[1],

                                         1 - x[0]**2 + x[1]]),

             'jac' : lambda x: np.array([[-1.0, -2.0],

                                         [-2*x[0], -1.0],

                                         [-2*x[0], 1.0]])}

def eqf(x):
    return jnp.array([2*x[0] + x[1] - 1])
    
def jeq(x):
    return np.array(jacobian(eqf)(x))
    

eq_cons = {'type': 'eq',

           'fun' : lambda x: np.array([2*x[0] + x[1] - 1]),

           #'jac' : lambda x: np.array([2.0, 1.0])}
           'jac' : lambda x: np.array(jeq(x))}

x0 = np.array([0.5, 0])
print("1: ",jeq(x0))
res = minimize(rosen, x0, method='SLSQP', jac=rosen_der,

              constraints=[eq_cons], options={'ftol': 1e-9, 'disp': True})

print(res)

1:  [[2. 1.]]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3427175748433131
            Iterations: 5
            Function evaluations: 9
            Gradient evaluations: 5
     fun: 0.3427175748433131
     jac: array([-0.82696629, -0.41348338])
 message: 'Optimization terminated successfully'
    nfev: 9
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.41494432, 0.17011137])


In [7]:

def rosen(x):

    """The Rosenbrock function"""

    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def hessian(f):
    return jacfwd(jacrev(f))

def jacobian(f):
    return jacfwd(f)

df=jacobian(rosen)

def c(x):
    return 2*x[0]+x[1]-1

A=jacobian(c)
C=hessian(c)
W=hessian(rosen)

x0 = np.array([0.5, 0])

def phi(x,lamda):
    return np.linalg.norm((df(x)-A(x)*lamda),ord=2)**2+c(x)**2
beta=0.8
epsi=1e-9
xk=x0
lamdak=1

def left(x,lamda):
    return np.vstack((np.hstack((W(x)-lamda*C(x),np.reshape(-A(x),(2,1)))),np.hstack((np.reshape(-A(x),(1,2)),np.reshape([0],(1,1))))))

def right(x,lamda):
    return -np.vstack((np.reshape(df(x),(2,1))-lamda*np.reshape(A(x),(2,1)),-np.reshape(c(x),(1,1))))

while phi(xk,lamdak)>epsi:
    dx=np.reshape(np.linalg.solve(left(xk,lamdak),right(xk,lamdak)),(3))
    alpha=1
    xkk=xk+alpha*np.array([dx[0],dx[1]])
    lamdakk=lamdak+alpha*dx[2]
    while(phi(xkk,lamdakk)>(1-beta*alpha)*phi(xk,lamdak)):
        alpha=alpha/4
        xkk=xk+alpha*np.array([dx[0],dx[1]])
        lamdakk=lamdak+alpha*dx[2]
    xk=xkk
    lamdak=lamdakk
#dx=np.reshape(np.linalg.solve(left(xk,lamdak),right(xk,lamdak)),(3))
#print(np.reshape(dx,(3)))
#print(xk+alpha*np.array([dx[0],dx[1]]))
print(xk,lamdak,phi(xk,lamdak))

[0.41494432 0.17011135] -0.41348410536118446 2.4584779439852858e-11


In [8]:
W=jax.hessian(f)
g=jacfwd(f)

In [11]:
def constrain_diff(*args):
        args=list(args)
        cons_jac=lambda x: np.array(jax.jacfwd(args[0])(x))
        cons_hess=lambda x:np.array(jax.hessian(args[0])(x))
        return [cons_jac,cons_hess]
def genW(f_W,cons_hess_W,x0_W):
        len_hess_W=len(cons_hess_W(x0_W))
        f_jac_W=lambda x:np.array(jax.hessian(f_W)(x))
        def W(x_W,lmd_W):
            W_=f_jac_W(x_W)
            for i_W in range(len_hess_W):
                W_-=lmd_W[i_W]*cons_hess_W(x_W)[i_W]
            return W_
        return W
# d is a row vector
# this function compute the objective of subproblem
# for given W_k and g_k
def obj(Wk,gk):
    return lambda d:0.5*np.dot(d,np.dot(Wk,d.T))+np.dot(gk,d)
# L1 penal function
def P(f_P,sigma_P,eq_P,ineq_P):
    return lambda x:f_P(x)+sigma_P*(np.sum(eq_P[0](x))+np.sum(np.abs(ineq_P[0](x))>0))
def gen_cons(cons_,cons_jac_,xk_):
    return lambda d:cons_(xk_)+np.dot(xk_,d)
def sqp_ker(f,eq,ineq,x0,epsi=1e-9,sigma=1,rho=0.8):
    # diffrentiate the constrains
    [eq_jac_ker,eq_hess_ker]=constrain_diff(eq)
    [ineq_jac_ker,ineq_hess_ker]=constrain_diff(ineq)
    # generate matrix W and vector g
    W_ker=genW(f,lambda x: np.vstack((eq_hess_ker(x),ineq_hess_ker(x))),x0)
    g_ker=lambda d: np.array(jacfwd(f)(d))
    
    itr=0
    dk=np.inf
    while(np.linalg.norm(dk,ord=2)>epsi or itr==0):
        itr+=1
        objk=lambda d:obj(d,W(xk,lamdak),g(xk))
        jack=lambda d:np.array(jacfwd(objk)(d))
        hessk=lambda d:np.array(jax.hessian(objk)(d))
        dk0=dk
        res = minimize(objk, dk0, method='trust-constr', jac=jack, hess=hessk,
                      constraints=constr(eq,ineq),
                      options={'verbose': 0})
        alphak=1
        one_dim_search(P,)
        xk=xk+alphak*dk
# test genW and cons_diff
[eq_jac_ker,eq_hess_ker]=constrain_diff(cons_f, 1, 1)
[ineq_jac_ker,ineq_hess_ker]=constrain_diff(cons_f, 1, 1)
#print(len(np.vstack((eq_hess_ker(x0),ineq_hess_ker(x0)))))
W_ker=genW(f,lambda x:np.vstack((eq_hess_ker(x),ineq_hess_ker(x))),x0)   
#print(W_ker(x0,np.array([1,1,1,1])))
g_ker=obj([[1,1],[2,2]],[1,1])(np.array([1,1]))
#print([cons_f, 1, 1][0](np.array([1,1])))
P_ker=P(rosen,1,[cons_f, 1, 1],[cons_f, 1, 1])
print(P_ker(np.array([1,1])))

3.0


In [10]:
from scipy.optimize import Bounds

bounds = Bounds([0, -0.5], [1.0, 2.0])
def rosen(x):

    """The Rosenbrock function"""

    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
def rosen_hess(x):

    return np.array(jax.hessian(rosen)(x))

from scipy.optimize import LinearConstraint

linear_constraint = LinearConstraint([2, 1], [1], [1])

def cons_f(x):

    return [x[0]**2 + x[1], x[0]**2 - x[1]]

def cons_J(x):

    return [[2*x[0], 1], [2*x[0], -1]]

def cons_H(x, v):

    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])

from scipy.optimize import NonlinearConstraint

def func2(*args):
    args=list(args)
    length=len(jacfwd(args[0])(args[5]))
    a=0*args[5]
    for i in range(length):
        print(i)
        a=a+jacfwd(args[0])(args[5])[i]
        
    print(a)
    print(jax.hessian(args[0])(args[5]))
    args.pop()
    return NonlinearConstraint(*args)
cj=lambda x: np.array(jax.jacfwd(cons_f)(x))
def ch(x,v):
    hess=np.array(jax.hessian(cons_f)(x))
    length=len(hess)
    ret=0*hess[0]
    for i in range(length):
        ret=ret+v[i]*hess[i]
    return ret
#nonlinear_constraint = NonlinearConstraint(cons_f, 1, 1, jac=cons_J, hess=cons_H)

nonlinear_constraint=func2(cons_f, 1, 1,cj,ch,x0)
x0 = np.array([0.5, 0])

res = minimize(rosen, x0, method='trust-constr', jac=rosen_der, hess=rosen_hess,

               constraints=[nonlinear_constraint],

               options={'verbose': 1})
print(res.x)
print(rosen_hess(x0))

0
1
[2. 0.]
[DeviceArray([[2., 0.],
             [0., 0.]], dtype=float32), DeviceArray([[2., 0.],
             [0., 0.]], dtype=float32)]
`gtol` termination condition is satisfied.
Number of iterations: 10, function evaluations: 10, CG iterations: 0, optimality: 0.00e+00, constraint violation: 9.08e-11, execution time: 0.45 s.
[ 1.00000000e+00 -1.27445239e-12]
[[ 302. -200.]
 [-200.  200.]]


In [ ]:
a=np.array([1,1])
b=np.array([3,4])
a-=b
print(a)

In [ ]:
print(np.dot(rosen_der(np.array([1.,2.])),np.array([1,1])))

In [ ]:
def nigger(a,b,c,d):
    return a+b+c+d
e=[1,2,3,4]
print(nigger(*e))

In [ ]:
a=lambda x,y:x**2+y
print(a(5,2))

In [ ]:
print(np.sum(np.array([-1,-1])>0))

In [4]:
print(cons_f(np.array([-1,-1]))+np.dot(cons_J(np.array([-1,-1])),np.array([-1,-1])))

NameError: name 'cons_f' is not defined

In [30]:
x0=np.array([1.,1.])
def f(x):
    return -jnp.sin(x[0])*jnp.cos(x[1])+jnp.sin(x[1])*jnp.cos(x[0])
    
jac_f=lambda x:np.array(jacfwd(f)(x))
hess_f=lambda x:np.array(hessian(f)(x))

def ineq(x):
    return [x[0]+x[1],-x[0]-x[1]-jnp.exp(-7*x[0])+jnp.pi]
jac_ineq=lambda x:np.array(jacfwd(ineq)(x))
hess_ineq=lambda x:np.array(hessian(ineq)(x))

def eq(x):
    return [x[0]-x[1]**3]
jac_eq=lambda x:np.array(jacfwd(eq)(x))
hess_eq=lambda x:np.array(hessian(eq)(x))

def genW(f_W,cons_hess_W,x0_W):
        len_hess_W=len(cons_hess_W(x0_W))
        f_jac_W=lambda x:np.array(jax.hessian(f_W)(x))
        def W(x_W,lmd_W):
            W_=f_jac_W(x_W)
            for i_W in range(len_hess_W):
                W_-=lmd_W[i_W]*cons_hess_W(x_W)[i_W]
            return W_
        return W
# d is a row vector
# this function compute the objective of subproblem
# for given W_k and g_k
def obj(Wk,gk):
    return lambda d:0.5*np.dot(d,np.dot(Wk,d.T))+np.dot(gk,d)
# L1 penal function
def P(f_P,sigma_P,eq_P,ineq_P):
    return lambda x:f_P(x)+sigma_P*(np.sum(np.abs(eq_P[0](x)))+np.sum(np.maximum(ineq_P[0](x),0,out=None)))
def gen_cons(cons_,cons_jac_,xk_):
    return lambda d:cons_(xk_)+np.dot(xk_,d)
print(genW(hess_f,lambda x: np.vstack((hess_eq(x),hess_ineq(x))),x0))
print(P(f,1,[eq,1,1],[ineq,1,1])(x0))
print((np.maximum(ineq(x0),0,out=None)))

<function genW.<locals>.W at 0x7fcfac467280>
2.0
[2.         1.14068079]


3.141592653589793
